In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv
load_dotenv()
DOWNLOAD_DIR = os.path.join(os.getcwd(), "downloads")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
}
options.add_experimental_option("prefs", prefs)
options.binary_location = os.getenv("binary_location")

# 1) 드라이버 셋업
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 반복할 키워드 목록
keywords = ['green', 'kb맑은']

try:
    # 기본 URL 로드
    base_url = os.getenv("base_url")
    driver.get(base_url)
    print('[LOG] 기본 페이지 로드 완료')

    for kw_text in keywords:
        print(f"[LOG] === 키워드: {kw_text} 검색 시작 ===")


        kw_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'keyword'))
        )

        kw_input.click()
        kw_input.clear()
        kw_input.send_keys(kw_text)
        print(f"[LOG] 검색어 “{kw_text}” 입력 완료")


        driver.execute_script(
            "document.getElementById('keyword').value = arguments[0];"
            "$('.midGb').find('.initData').val('');"
            "goSearchBefore();",
            kw_text
        )
        print('[LOG] goSearchBefore() 호출 완료')


        WebDriverWait(driver, 10).until(EC.url_contains('page=C'))
        WebDriverWait(driver, 10).until(
            lambda d: len(d.find_elements(By.CSS_SELECTOR, '.midGb .list li')) > 0
        )
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#formList li"))
        )
        print('[LOG] 검색 결과 로드 완료')

        # 다운로드 링크 수집
        download_links = driver.find_elements(
            By.CSS_SELECTOR,
            "#formList a[href^='javascript:OpenStreamDownload']"
        )
        print(f"[LOG] 다운로드 링크 {len(download_links)}개 발견")

        # 순차 클릭
        for idx, link in enumerate(download_links, 1):
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", link)
            driver.execute_script("arguments[0].click();", link)
            title = link.get_attribute('title')
            time.sleep(2)


        driver.get(base_url)

        time.sleep(2)

    print('[LOG] 모든 키워드 처리 완료')

finally:
    driver.quit()


[LOG] 기본 페이지 로드 완료
[LOG] === 키워드: green 검색 시작 ===
[LOG] 검색어 “green” 입력 완료
[LOG] goSearchBefore() 호출 완료
[LOG] 검색 결과 로드 완료
[LOG] 다운로드 링크 6개 발견
[LOG] === 키워드: kb맑은 검색 시작 ===
[LOG] 검색어 “kb맑은” 입력 완료
[LOG] goSearchBefore() 호출 완료
[LOG] 검색 결과 로드 완료
[LOG] 다운로드 링크 6개 발견
[LOG] 모든 키워드 처리 완료
